In [1]:
print('Installing torchprofile...')
!pip install torchprofile 1>/dev/null

Installing torchprofile...


In [2]:
import copy
import math
import random
from collections import OrderedDict, defaultdict

from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *

from torchprofile import profile_macs

assert torch.cuda.is_available()

/usr/lib/python3.10/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [13]:
class VGG(nn.Module):
    ARCH = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']

    def __init__(self) -> None:
        super().__init__()

        layers = []
        counts = defaultdict(int)

        def add(name: str, layer: nn.Module) -> None:
            layers.append((f"{name}{counts[name]}", layer))
            counts[name] += 1

        in_channels = 3
        for x in self.ARCH:
            if x != 'M':
                # conv-bn-relu
                add("conv", nn.Conv2d(in_channels, x, 3, padding=1, bias=False))
                add("bn", nn.BatchNorm2d(x))
                add("relu", nn.ReLU(True))
                in_channels = x
            else:
                # maxpool
                add("pool", nn.MaxPool2d(2))
        add("avgpool", nn.AvgPool2d(1))
        self.backbone = nn.Sequential(OrderedDict(layers))
        self.classifier = nn.Linear(512, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.backbone(x)
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x

In [30]:
def train( model: nn.Module, 
          dataloader: DataLoader, 
          criterion: nn.Module, 
          optimizer: Optimizer, 
          scheduler: LambdaLR, 
          callbacks = None) -> None:
    model.train()

    for inputs, targets in tqdm(dataloader, desc='train', leave=False):
        # Move the data from CPU to GPU
        inputs = inputs.cuda()
        targets = targets.cuda()

        # Reset the gradients (from the last iteration)
        optimizer.zero_grad()

        # Forward inference
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward propagation
        loss.backward()

        # Update optimizer and LR scheduler
        optimizer.step()
        scheduler.step()

        if callbacks is not None:
            for callback in callbacks:
                callback()

In [31]:
@torch.inference_mode()
def evaluate( model: nn.Module,
              dataloader: DataLoader,
              extra_preprocess = None ) -> float:
    model.eval()

    num_samples = 0
    num_correct = 0

    for inputs, targets in tqdm(dataloader, desc="eval", leave=False):
        # Move the data from CPU to GPU
        inputs = inputs.cuda()
        if extra_preprocess is not None:
            for preprocess in extra_preprocess:
                inputs = preprocess(inputs)

        targets = targets.cuda()

        # Inference
        outputs = model(inputs)

        # Convert logits to class indices
        outputs = outputs.argmax(dim=1)

        # Update metrics
        num_samples += targets.size(0)
        num_correct += (outputs == targets).sum()

    return (num_correct / num_samples * 100).item()

In [32]:
import os

image_size = 32

# Define the transformations
transforms = {
    "train": Compose([
        RandomCrop(image_size, padding=4),
        RandomHorizontalFlip(),
        ToTensor(),
    ]),
    "test": ToTensor(),
}

# Get the base path for the data folder (assuming 'Quantization' is the main folder)
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data')

# Load the datasets
dataset = {}
for split in ["train", "test"]:
    dataset[split] = CIFAR10(
        root=os.path.join(data_dir, 'cifar10'),  # Use the relative path to the data folder
        train=(split == "train"),
        download=True,
        transform=transforms[split],
    )

# Create the dataloaders
dataloader = {}
for split in ['train', 'test']:
    dataloader[split] = DataLoader(
        dataset[split],
        batch_size=512,
        shuffle=(split == 'train'),
        num_workers=0,
        pin_memory=True,
    )

Files already downloaded and verified
Files already downloaded and verified


In [33]:
import torch.optim as optim
# Model, criterion, optimizer

model = VGG().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: 0.95 ** epoch)  # Decays LR by 0.95 every epoch


In [37]:
# Training for 20 epochs
epochs = 200
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train(model, dataloader["train"], criterion, optimizer, scheduler)

test_acc = evaluate(model, dataloader['test'])
print(f"Test Acc: {test_acc}")

Epoch 1/200


Epoch 2/200


Epoch 3/200


Epoch 4/200


Epoch 5/200


...

Epoch 197/200


Epoch 198/200


Epoch 199/200


Epoch 200/200


Test Acc: 31.119998931884766
